In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df_airlines = pd.read_csv('./data/airlines.csv')
df_airports = pd.read_csv('./data/airports.csv')
df_future = pd.read_csv('./data/future_data.csv')
df_historic = pd.read_csv('./data/historic_data.csv')

## Variables and Data Type

useless airllines

In [97]:
df_airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


data types

In [98]:
df_airports.dtypes

IATA_CODE     object
AIRPORT       object
CITY          object
STATE         object
COUNTRY       object
LATITUDE     float64
LONGITUDE    float64
dtype: object

In [146]:
df_future.dtypes

SCHEDULED_DEPARTURE    object
SCHEDULED_ARRIVAL      object
AIRLINE                object
FLIGHT_NUMBER           int64
TAIL_NUMBER            object
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
DISTANCE                int64
dtype: object

In [147]:
df_historic.dtypes

SCHEDULED_DEPARTURE     object
DEPARTURE_TIME          object
SCHEDULED_ARRIVAL       object
ARRIVAL_TIME            object
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

## No actual missing value except tail number

- delay cols are NA means 0
- depature_time taxi_out .... are NA means cancelled

In [90]:
def check_nan(df):
    cols1 = []
    for i in range(df_historic.shape[1]):
        col = df.columns[i]
        if(df[col].isna().sum()/df[col].shape[0] > 0):
            cols1.append(col)
            print("cols：",df.columns[i])
    print("number of cols：",len(cols1))
    df = df.drop(cols1,1)

In [91]:
check_nan(df_historic)

cols： DEPARTURE_TIME
cols： ARRIVAL_TIME
cols： TAIL_NUMBER
cols： DEPARTURE_DELAY
cols： TAXI_OUT
cols： WHEELS_OFF
cols： SCHEDULED_TIME
cols： ELAPSED_TIME
cols： AIR_TIME
cols： WHEELS_ON
cols： TAXI_IN
cols： CANCELLATION_REASON
cols： AIR_SYSTEM_DELAY
cols： SECURITY_DELAY
cols： AIRLINE_DELAY
cols： LATE_AIRCRAFT_DELAY
cols： WEATHER_DELAY
number of cols： 17


In [4]:
df_historic[df_historic['TAIL_NUMBER'].isnull()]

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,ARRIVAL_TIME,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,...,DISTANCE,WHEELS_ON,TAXI_IN,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
37,2019-03-01 00:11:00,NaN,02:59:00,NaN,UA,1881,NaN,ORD,IAD,NaN,...,588,NaN,NaN,1,A,NaN,NaN,NaN,NaN,NaN
38,2019-03-01 00:11:00,NaN,03:38:00,NaN,UA,1489,NaN,DEN,MEM,NaN,...,872,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
41,2019-03-01 00:14:00,NaN,03:14:00,NaN,UA,1610,NaN,DEN,MSP,NaN,...,680,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
43,2019-03-01 00:14:00,NaN,03:21:00,NaN,UA,1083,NaN,ORD,ORF,NaN,...,717,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
60,2019-03-01 00:18:00,NaN,05:17:00,NaN,UA,1492,NaN,DEN,PIT,NaN,...,1290,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457882,2019-07-31 21:40:00,NaN,23:56:00,NaN,UA,479,NaN,IAH,SFO,NaN,...,1635,NaN,NaN,1,C,NaN,NaN,NaN,NaN,NaN
2458126,2019-07-31 22:00:00,NaN,23:28:00,NaN,US,2138,NaN,DCA,BOS,NaN,...,399,NaN,NaN,1,C,NaN,NaN,NaN,NaN,NaN
2458186,2019-07-31 22:06:00,NaN,01:24:00,NaN,UA,1454,NaN,DEN,IAH,NaN,...,862,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN
2458192,2019-07-31 22:08:00,NaN,23:59:00,NaN,UA,1088,NaN,IAD,DEN,NaN,...,1452,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN


## Feature Selection

Only consider *total depature delay*, drop other delays

Fill dely with a big number if cancelled

df_airlines does not provide any extra info, so not use it

df_airport provide extra info about location
- Airport extra info: city, state, latitue, longitute
- Only use latitue, longitute is sufficent (temporally)

### cols in Test Data

- replace TAIL_NUMBER, FLIGHT_NUMBER with mean delay time in history data
- drop AIRPORT, replace it with LATITUDE LONGITUDE

In [71]:
drop_list = ['SCHEDULED_ARRIVAL']
df_test = df_future.drop(drop_list,axis=1)

mean delay of tail/flight

In [73]:
df_delay_tail = df_historic.groupby(['TAIL_NUMBER']).mean()['DEPARTURE_DELAY']
df_delay_flight = df_historic.groupby(['FLIGHT_NUMBER']).mean()['DEPARTURE_DELAY']

In [74]:
df_delay_tail = pd.DataFrame(df_delay_tail)
df_delay_flight = pd.DataFrame(df_delay_flight)

replace tail/flight number with avarage delay, but not work

In [80]:
df_delay_tail['TAIL_NUMBER'] = df_delay_tail.index
tmp = pd.DataFrame(df_delay_tail.values)
tmp.columns = ['TAIL_DELAY','TAIL_NUMBER']

In [83]:
df_test = pd.merge(df_test,tmp.loc[:,['TAIL_NUMBER','TAIL_DELAY']],how='left',on = 'TAIL_NUMBER').drop(['TAIL_NUMBER'],axis=1)

In [87]:
df_delay_flight['FLIGHT_NUMBER'] = df_delay_flight.index
tmp = pd.DataFrame(df_delay_flight.values)
tmp.columns = ['FLIGHT_DELAY','FLIGHT_NUMBER']

In [90]:
df_test = pd.merge(df_test,tmp.loc[:,['FLIGHT_NUMBER','FLIGHT_DELAY']],how='left',on = 'FLIGHT_NUMBER').drop(['FLIGHT_NUMBER'],axis=1)

In [91]:
df_test

,SCHEDULED_DEPARTURE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,TAIL_DELAY,FLIGHT_DELAY
0,2022-08-01 00:05:00,AA,LAX,DFW,1235,10.5982,8.180288
1,2022-08-01 00:15:00,US,PHX,CLT,1773,9.39557,9.296167
2,2022-08-01 00:15:00,F9,PHX,CVG,1569,14.0622,9.307285
3,2022-08-01 00:20:00,AA,LAX,MIA,2342,10.685,8.417845
4,2022-08-01 00:25:00,NK,LAS,MSP,1299,11.755,12.705556
...,...,...,...,...,...,...,...
469963,2022-08-31 23:59:00,B6,JFK,BQN,1576,14.121,17.121069
469964,2022-08-31 23:59:00,DL,SEA,DTW,1927,9,7.183099
469965,2022-08-31 23:59:00,F9,DEN,TPA,1506,15.9387,17.103448
469966,2022-08-31 23:59:00,F9,DEN,ATL,1199,10.745,8.225312


### cols in Training Data

The extra feature in df_historic which df_future does not containe will be droped:
- TAXI_OUT
- WHEELS_OFF
- SCHEDULED_TIME
- ELAPSED_TIME
- AIR_TIME     
- WHEELS_ON
- TAXI_IN  
- ...

- drop DEPARTURE_TIME, because we have DEPARTURE_DELAY and SCHEDULED_DEPARTURE

In [99]:
df_train = df_historic.drop(['TAXI_OUT', 'WHEELS_OFF','SCHEDULED_TIME','ELAPSED_TIME','AIR_TIME','WHEELS_ON', 'TAXI_IN','SCHEDULED_ARRIVAL','ARRIVAL_TIME','DEPARTURE_TIME','CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY','CANCELLED'], axis=1)

Fill big delay number when cancelled == 1

In [17]:
df_train['DEPARTURE_DELAY'].max()

1878.0

In [100]:
df_train['DEPARTURE_DELAY'] = df_train['DEPARTURE_DELAY'].fillna(1e5)

In [101]:
df_train

,SCHEDULED_DEPARTURE,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE
0,2019-03-01 00:01:00,UA,1238,N812UA,DEN,BIL,1.0,455
1,2019-03-01 00:01:00,EV,6134,N14171,ORD,BNA,28.0,409
2,2019-03-01 00:01:00,EV,4682,N15910,ORD,BHM,100000.0,584
3,2019-03-01 00:02:00,UA,1550,N822UA,ORD,BTV,-3.0,763
4,2019-03-01 00:02:00,UA,1502,N33262,ORD,BOS,34.0,867
...,...,...,...,...,...,...,...,...
2458623,2019-07-31 23:59:00,AA,1496,N3MJAA,LAS,ORD,-5.0,1514
2458624,2019-07-31 23:59:00,AA,1283,N3GEAA,SEA,ORD,51.0,1721
2458625,2019-07-31 23:59:00,AA,219,N3LKAA,LAX,ORD,50.0,1744
2458626,2019-07-31 23:59:00,B6,1503,N592JB,JFK,SJU,84.0,1598


Replace tail/flight number with mean delay

In [102]:
tmp = pd.DataFrame(df_delay_tail.values)
tmp.columns = ['TAIL_DELAY','TAIL_NUMBER']
df_train = pd.merge(df_train,tmp.loc[:,['TAIL_NUMBER','TAIL_DELAY']],how='left',on = 'TAIL_NUMBER').drop(['TAIL_NUMBER'],axis=1)

In [104]:
tmp = pd.DataFrame(df_delay_flight.values)
tmp.columns = ['FLIGHT_DELAY','FLIGHT_NUMBER']
df_train = pd.merge(df_train,tmp.loc[:,['FLIGHT_NUMBER','FLIGHT_DELAY']],how='left',on = 'FLIGHT_NUMBER').drop(['FLIGHT_NUMBER'],axis=1)

In [107]:
df_train

,SCHEDULED_DEPARTURE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,TAIL_DELAY,FLIGHT_DELAY
0,2019-03-01 00:01:00,UA,DEN,BIL,1.0,455,18.5846,15.515845
1,2019-03-01 00:01:00,EV,ORD,BNA,28.0,409,9.3525,1.450000
2,2019-03-01 00:01:00,EV,ORD,BHM,100000.0,584,10.8817,10.876543
3,2019-03-01 00:02:00,UA,ORD,BTV,-3.0,763,16.4404,11.677419
4,2019-03-01 00:02:00,UA,ORD,BOS,34.0,867,25.0573,11.158863
...,...,...,...,...,...,...,...,...
2458623,2019-07-31 23:59:00,AA,LAS,ORD,-5.0,1514,7,13.922330
2458624,2019-07-31 23:59:00,AA,SEA,ORD,51.0,1721,8.58864,13.212171
2458625,2019-07-31 23:59:00,AA,LAX,ORD,50.0,1744,5.70575,10.814849
2458626,2019-07-31 23:59:00,B6,JFK,SJU,84.0,1598,15.6675,13.843972


## Feature converting

### Time data process

In [108]:
def time_process(df):
    #df_test['Year'] = pd.to_datetime(df_test['SCHEDULED_DEPARTURE']).dt.year
    #df_test['Month'] = pd.to_datetime(df_test['SCHEDULED_DEPARTURE']).dt.month
    df['Day'] = pd.to_datetime(df['SCHEDULED_DEPARTURE']).dt.day
    df['SCHEDULED_DEPARTURE_Time'] = df['SCHEDULED_DEPARTURE'].apply(lambda x:x[-8:])
    df['Week_Day'] = pd.to_datetime(df['SCHEDULED_DEPARTURE']).dt.weekday
    df['Zero'] = '00:00:00'
    df['Dep_Time'] = (pd.to_datetime(df['SCHEDULED_DEPARTURE_Time']) - pd.to_datetime(df['Zero'])).apply(lambda x:x.seconds)
    df.drop(['Zero','SCHEDULED_DEPARTURE_Time','SCHEDULED_DEPARTURE'],axis=1,inplace=True)
    return df

In [109]:
df_test = time_process(df_test)

In [110]:
df_test

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,TAIL_DELAY,FLIGHT_DELAY,Day,Week_Day,Dep_Time
0,AA,LAX,DFW,1235,10.5982,8.180288,1,0,300
1,US,PHX,CLT,1773,9.39557,9.296167,1,0,900
2,F9,PHX,CVG,1569,14.0622,9.307285,1,0,900
3,AA,LAX,MIA,2342,10.685,8.417845,1,0,1200
4,NK,LAS,MSP,1299,11.755,12.705556,1,0,1500
...,...,...,...,...,...,...,...,...,...
469963,B6,JFK,BQN,1576,14.121,17.121069,31,2,86340
469964,DL,SEA,DTW,1927,9,7.183099,31,2,86340
469965,F9,DEN,TPA,1506,15.9387,17.103448,31,2,86340
469966,F9,DEN,ATL,1199,10.745,8.225312,31,2,86340


In [111]:
df_train = time_process(df_train)

In [112]:
df_train

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,TAIL_DELAY,FLIGHT_DELAY,Day,Week_Day,Dep_Time
0,UA,DEN,BIL,1.0,455,18.5846,15.515845,1,4,60
1,EV,ORD,BNA,28.0,409,9.3525,1.450000,1,4,60
2,EV,ORD,BHM,100000.0,584,10.8817,10.876543,1,4,60
3,UA,ORD,BTV,-3.0,763,16.4404,11.677419,1,4,120
4,UA,ORD,BOS,34.0,867,25.0573,11.158863,1,4,120
...,...,...,...,...,...,...,...,...,...,...
2458623,AA,LAS,ORD,-5.0,1514,7,13.922330,31,2,86340
2458624,AA,SEA,ORD,51.0,1721,8.58864,13.212171,31,2,86340
2458625,AA,LAX,ORD,50.0,1744,5.70575,10.814849,31,2,86340
2458626,B6,JFK,SJU,84.0,1598,15.6675,13.843972,31,2,86340


### Airport info

In [113]:
df_test = df_test.join(df_airports.set_index('IATA_CODE'),on='ORIGIN_AIRPORT').drop(['ORIGIN_AIRPORT','AIRPORT','COUNTRY','CITY','STATE'], axis = 1)

In [115]:
df_train = df_train.join(df_airports.set_index('IATA_CODE'),on='ORIGIN_AIRPORT').drop(['ORIGIN_AIRPORT','AIRPORT','CITY','STATE'], axis = 1)

In [116]:
df_train

,AIRLINE,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,TAIL_DELAY,FLIGHT_DELAY,Day,Week_Day,Dep_Time,COUNTRY,LATITUDE,LONGITUDE
0,UA,BIL,1.0,455,18.5846,15.515845,1,4,60,USA,39.85841,-104.66700
1,EV,BNA,28.0,409,9.3525,1.450000,1,4,60,USA,41.97960,-87.90446
2,EV,BHM,100000.0,584,10.8817,10.876543,1,4,60,USA,41.97960,-87.90446
3,UA,BTV,-3.0,763,16.4404,11.677419,1,4,120,USA,41.97960,-87.90446
4,UA,BOS,34.0,867,25.0573,11.158863,1,4,120,USA,41.97960,-87.90446
...,...,...,...,...,...,...,...,...,...,...,...,...
2458623,AA,ORD,-5.0,1514,7,13.922330,31,2,86340,USA,36.08036,-115.15233
2458624,AA,ORD,51.0,1721,8.58864,13.212171,31,2,86340,USA,47.44898,-122.30931
2458625,AA,ORD,50.0,1744,5.70575,10.814849,31,2,86340,USA,33.94254,-118.40807
2458626,B6,SJU,84.0,1598,15.6675,13.843972,31,2,86340,USA,40.63975,-73.77893


### One-hot encoding

In [119]:
def one_hot_encoding(df,col):
    return pd.concat([df,pd.get_dummies(df[col])],axis=1).drop([col],axis=1)

In [120]:
df_test = one_hot_encoding(df_test, 'AIRLINE')
# df_test = one_hot_encoding(df_test, 'CITY')
# df_test = one_hot_encoding(df_test, 'STATE')

In [121]:
df_test

,DESTINATION_AIRPORT,DISTANCE,TAIL_DELAY,FLIGHT_DELAY,Day,Week_Day,Dep_Time,LATITUDE,LONGITUDE,AA,...,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
0,DFW,1235,10.5982,8.180288,1,0,300,33.94254,-118.40807,1,...,0,0,0,0,0,0,0,0,0,0
1,CLT,1773,9.39557,9.296167,1,0,900,33.43417,-112.00806,0,...,0,0,0,0,0,0,0,1,0,0
2,CVG,1569,14.0622,9.307285,1,0,900,33.43417,-112.00806,0,...,0,1,0,0,0,0,0,0,0,0
3,MIA,2342,10.685,8.417845,1,0,1200,33.94254,-118.40807,1,...,0,0,0,0,0,0,0,0,0,0
4,MSP,1299,11.755,12.705556,1,0,1500,36.08036,-115.15233,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469963,BQN,1576,14.121,17.121069,31,2,86340,40.63975,-73.77893,0,...,0,0,0,0,0,0,0,0,0,0
469964,DTW,1927,9,7.183099,31,2,86340,47.44898,-122.30931,0,...,0,0,0,0,0,0,0,0,0,0
469965,TPA,1506,15.9387,17.103448,31,2,86340,39.85841,-104.66700,0,...,0,1,0,0,0,0,0,0,0,0
469966,ATL,1199,10.745,8.225312,31,2,86340,39.85841,-104.66700,0,...,0,1,0,0,0,0,0,0,0,0


In [122]:
df_train = one_hot_encoding(df_train, 'AIRLINE')
# df_train = one_hot_encoding(df_train, 'CITY')
# df_train = one_hot_encoding(df_train, 'STATE')

In [123]:
df_train

,DESTINATION_AIRPORT,DEPARTURE_DELAY,DISTANCE,TAIL_DELAY,FLIGHT_DELAY,Day,Week_Day,Dep_Time,COUNTRY,LATITUDE,...,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
0,BIL,1.0,455,18.5846,15.515845,1,4,60,USA,39.85841,...,0,0,0,0,0,0,1,0,0,0
1,BNA,28.0,409,9.3525,1.450000,1,4,60,USA,41.97960,...,1,0,0,0,0,0,0,0,0,0
2,BHM,100000.0,584,10.8817,10.876543,1,4,60,USA,41.97960,...,1,0,0,0,0,0,0,0,0,0
3,BTV,-3.0,763,16.4404,11.677419,1,4,120,USA,41.97960,...,0,0,0,0,0,0,1,0,0,0
4,BOS,34.0,867,25.0573,11.158863,1,4,120,USA,41.97960,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458623,ORD,-5.0,1514,7,13.922330,31,2,86340,USA,36.08036,...,0,0,0,0,0,0,0,0,0,0
2458624,ORD,51.0,1721,8.58864,13.212171,31,2,86340,USA,47.44898,...,0,0,0,0,0,0,0,0,0,0
2458625,ORD,50.0,1744,5.70575,10.814849,31,2,86340,USA,33.94254,...,0,0,0,0,0,0,0,0,0,0
2458626,SJU,84.0,1598,15.6675,13.843972,31,2,86340,USA,40.63975,...,0,0,0,0,0,0,0,0,0,0
